# Trabajo práctico N°1: FIUFIP

## Parte 1 - Análisis de datos
- Continanza, Nicolás (97576)
- De Angelis Riva, Lukas Nahuel  (103784)

#### FIUFIP: Que No Se Escape Nadie (?)

A partir de la información recolectada por FIUFIP, queremos hacer un análisis que nos permita inferir qué ciudadanos tienen un alto poder adquisitivo. Para esto, y tomando en cuenta la información con la que contamos y sus limitaciones, creemos que será valioso poder responder las siguientes preguntas, y evaluar si nos aportan información útil:

1. ¿Cuántas personas tienen un alto poder adquisitivo en los datos recolectados por la agencia?
2. ¿Cómo se relaciona cada una de las variables del dataset con el poder adquisitivo de las personas presentes en la muestra?
3. ¿Qué variables deberíamos tener en cuenta para clasificar a las personas muestreadas según su poder adquisitivo?

## Bibliotecas

Comencemos por importar las bibliotecas a utilizar.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns
sns.set()
#import squarify
import numpy as np
import functools
import operator
import math

In [ ]:
def get_data():
    GSPREADHSEET_DOWNLOAD_URL = (
        "https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format
    )
    TP_GID = '1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0'
    df = pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=TP_GID))
    return df    

In [ ]:
df = get_data()

## Feature Engineering

Antes de comenzar el análisis exploratorio de los datos, estudiemos las características del dataset y analicemos si podemos hacer algunas mejoras para facilitar el trabajo.

### Missing Values

Comencemos por ver qué variables presentan valores nulos.

In [ ]:
df.isna().any()

Vemos que los únicos missing values se dan en las columnas: `barrio`, `categoría_de_trabajo` y `trabajo`.

Analicemos ahora la cantidad de personas cuyo barrio desconocemos.

In [ ]:
plt.figure(dpi=100)
ax = df['barrio'].notnull().apply(lambda x: 'Con barrio' if x else 'Sin barrio').value_counts().plot(
    kind='pie', autopct='%1.0f%%', colors=['#cc79a7', '#f0e442'], title = "Proporción missings en barrio")

ax.set_ylabel("")
plt.show()



Se trata de un 2% del dataset, un valor que consideramos lo suficientemente grande como para no descartarlo.

Veamos si hay grandes diferencias de poder adquisitivo entre las personas que no indicaron su barrio y las que si.

In [ ]:
df_test_barrio = df[['barrio', 'tiene_alto_valor_adquisitivo']].copy()
df_test_barrio['tiene_barrio'] = df_test_barrio['barrio'].isna().apply(lambda x: 'No' if x else 'Sí')
del df_test_barrio['barrio']

plt.figure(dpi=100)
ax = sns.barplot(data=df_test_barrio, x='tiene_barrio', y='tiene_alto_valor_adquisitivo', palette='rainbow', estimator=lambda x: np.mean(x)*100.0)

plt.ylabel("% Tiene alto valor adquisitivo")
plt.xlabel("Tiene barrio")
plt.title('Diferencia de poder adquisitivo entre gente con barrio y gente sin barrio')
plt.show()

Vemos que, a priori, haber informado el barrio o no haberlo hecho no tiene un impacto significativo sobre la variable objetivo. Por lo tanto, rellenaremos esos valores faltantes asignando la moda, siendo esta Palermo como se puede observar:

In [ ]:
df['barrio'].value_counts()[0:1]

In [ ]:
data_set_mejorado = df.copy()
data_set_mejorado['barrio'] = data_set_mejorado['barrio'].apply(lambda x: 'Palermo' if str(x) == 'nan' else x)

De forma similar, analizaremos la variable `categoria_de_trabajo`.

In [ ]:
plt.figure(dpi=100)
ax = df['categoria_de_trabajo'].notnull().apply(lambda x: 'Respondió' if x else 'No respondió').value_counts().plot(
    kind='pie', autopct='%1.0f%%', colors=['#cc79a7', '#f0e442'], title = 'Personas con categoria de trabajo')
ax.set_ylabel("")
plt.show()


Nuevamente estamos frente a un caso donde no podemos despreciar aquellas muestras que no informen su categoría de trabajo.

In [ ]:
df_test_categoria = df[['categoria_de_trabajo', 'tiene_alto_valor_adquisitivo']].copy()
df_test_categoria['tiene_categoria'] = df_test_categoria['categoria_de_trabajo'].isna().apply(lambda x: 'No' if x else 'Sí')
del df_test_categoria['categoria_de_trabajo']

plt.figure(dpi=150)
ax = sns.barplot(data=df_test_categoria, x='tiene_categoria', y='tiene_alto_valor_adquisitivo', palette='rainbow', estimator=lambda x: np.mean(x)*100.0)

plt.ylabel("% Tiene alto valor adquisitivo")
plt.xlabel("Tiene categoría de trabajo")
plt.title('Diferencia de poder adquisitivo entre gente con categoría de trabajo y gente sin categoría de trabajo')
plt.show()

Aquí ya se puede ver información interesante: la gente que no responde su categoría de trabajo, en general tiene menor valor adquisitivo.

Veamos ahora la relación entre valores nulos en `trabajo` y valores nulos en `categoría_de_trabajo`.

In [ ]:
df[['categoria_de_trabajo', 'trabajo']].isna().apply(lambda column: column.value_counts()[True])

Hay una diferencia de 7 entradas, que serán las siguientes:

In [ ]:
df[df['categoria_de_trabajo'].notnull() & df['trabajo'].isna()]

Esto quiere decir que tener un valor NA en `categoria_de_trabajo` se corresponde con tener un valor NA  en `trabajo`, y si hay un NA en trabajo, salvo por estos últimos 7 casos, habrá un NA en `categoria_de_trabajo`.

En base a esto, vamos a crear una nueva categoría llamada "sin_categoria", para todas aquellas personas que tengan valor nulo en `categoria_de_trabajo`, y otra llamada "no_responde" para quienes tengan valor nulo en `trabajo`.

In [ ]:
data_set_mejorado['categoria_de_trabajo'] = data_set_mejorado['categoria_de_trabajo'].apply(lambda x: 'Sin categoria' if str(x) == 'nan' else x)

In [ ]:
data_set_mejorado['trabajo'] = data_set_mejorado['trabajo'].apply(lambda x: 'No responde' if str(x) == 'nan' else x)

A continuación estudiaremos si `educacion_alcanzada` tiene una relación fuerte con la columna `anios_estudiados`

In [ ]:
df[['anios_estudiados', 'educacion_alcanzada']].groupby('educacion_alcanzada').aggregate(set)

Veámoslo en un gráfico:

In [ ]:
plt.xticks(range(1,21))

df_tmp = data_set_mejorado.sort_values(by='anios_estudiados', ascending=False)
sns.scatterplot(data=df_tmp, x="anios_estudiados", y="educacion_alcanzada")
plt.title('Relación entre la cantidad de años estudiados y el nivel de educación alcanzada', size=16)
plt.ylabel("Educación Alcanzada")
plt.xlabel("Años Estudiados")

Como esperábamos, la columna `educacion_alcanzada` posee la misma información que `anios_estudiados`.

Una interpretacion de esto es que la gente que esta cargada en este set de datos nunca repitió un año escolar ni se atrasó en la carrera universitaria.

Proponemos eliminar la columna `educacion_alcanzada`, ya que es una variable categórica (más difícil de manipular) con un orden impuesto directamente por la columna `anios_estudiados`.

In [ ]:
del data_set_mejorado['educacion_alcanzada']

Para finalizar, notemos que en la columna `rol_familiar_registrado` las personas solteras están identificadas con el valor "soltero_a", y no hay una distinción por género de la persona. Por el contrario, las personas casadas pueden tener valor "casado" o "casada" según su género. Veamos si hay diferencias de poder adquisitivo entre los casados y las casadas.

In [ ]:
df_test_categoria = df[['rol_familiar_registrado', 'tiene_alto_valor_adquisitivo']].copy()
df_casadxs = df_test_categoria[df_test_categoria['rol_familiar_registrado'].isin(['casado', 'casada'])]

plt.figure(dpi=100)
ax = sns.barplot(data=df_casadxs, x='rol_familiar_registrado', y='tiene_alto_valor_adquisitivo', palette='rainbow', estimator=lambda x: np.mean(x)*100.0)

plt.ylabel("% Tiene alto valor adquisitivo")
plt.xlabel("Rol familiar registrado")
plt.title('Diferencia de poder adquisitivo entre casados y casadas')
plt.show()

Dado que la diferencia es muy pequeña y que ya tenemos la posibilidad de conocer el género de la persona gracias a la columna `genero`, podemos unificar y quedarnos con un solo valor.

In [ ]:
data_set_mejorado['rol_familiar_registrado'] = data_set_mejorado['rol_familiar_registrado'].apply(lambda x : 'casado' if x == 'casada' else x)

A continuación, una función que encapsula todo el trabajo realizado sobre el dataset inicial.

In [ ]:
def feature_engineering(df):
    # Missings en barrio
    data_set_mejorado = df.copy()
    data_set_mejorado['barrio'] = data_set_mejorado['barrio'].apply(lambda x: 'Palermo' if str(x) == 'nan' else x)

    # Missings en categoría de trabajo
    data_set_mejorado['categoria_de_trabajo'] = data_set_mejorado['categoria_de_trabajo'].apply(lambda x: 'Sin categoria' if str(x) == 'nan' else x)

    # Missings en trabajo
    data_set_mejorado['trabajo'] = data_set_mejorado['trabajo'].apply(lambda x: 'No responde' if str(x) == 'nan' else x)

    # Eliminación de el atributo eduación alcanzada
    del data_set_mejorado['educacion_alcanzada']

    # Juntamos casado y casada en una misma categoría.
    data_set_mejorado['rol_familiar_registrado'] = data_set_mejorado['rol_familiar_registrado'].apply(lambda x : 'casado' if x == 'casada' else x)
    
    return data_set_mejorado

## Análisis Exploratorio

Con lo anterior, obtenemos los datos, aplicamos la función de feature engineering y comenzamos con el análisis exploratorio para clasificar a las personas muestreadas.

In [ ]:
df = get_data()
df = feature_engineering(df)

Nuestra variable objetivo será `tiene_alto_valor_adquisitivo` una variable que puede tomar valores 0 ó 1.

Veamos cómo se distribuye el valor de esta variable entre los individuos de la muestra que vamos a analizar. Por tratarse de una variable dicotómica de la que solo nos interesa conocer su distribución, podemos usar un gráfico de torta.

In [ ]:
plt.figure(dpi=100)
df["tiene_alto_valor_adquisitivo"].apply(lambda x: 'Sí' if x==1 else 'No').value_counts().plot(
    kind='pie', autopct='%1.0f%%', colors=['#cc79a7', '#f0e442']
)
plt.title('¿Tiene alto valor adquisitivo?')
plt.ylabel("")
plt.show()

Como podemos observar, cerca de las tres cuartas partes de las personas muestreadas tienen un bajo poder adquisitivo. 

Para continuar el análisis, vamos a ver cómo se relaciona (si es que lo hacen) cada columna del dataset con la variable objetivo

In [ ]:
df.columns

### Edad

Comencemos por entender cómo se distribuyen los individuos muestreados según su edad.

In [ ]:
df_tmp = df[['edad']].copy()
plt.figure(dpi=100)
sns.displot(df_tmp, x='edad', kind="kde", bw_adjust=.4, fill=True)
plt.show()

Vemos que la mayor parte de la población que vamos a analizar se encuentra entre los 18 y los 60 años.

#### ¿Hay una relación entre la edad y el valor adquisitivo?
Para intentar responder esta pregunta, agruparemos a los individuos muestreados por su edad, y estudiaremos cuántas personas de cada grupo etario tienen alto o bajo poder adquisitivo.

Para facilitar el análisis y la visualización de los resultados, dichos grupos etarios serán bins de 10 años.

In [ ]:
df_tmp = df[['edad', 'tiene_alto_valor_adquisitivo']].copy()
df_tmp['rango_etario'] = pd.cut(df_tmp.edad, range(10, int(df_tmp['edad'].max()) + 10, 10))

# Configuro el grafico, armo el countplot y setteo labels varios
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, dpi=150, figsize=(6 * 2, 3.5 * 2))
sns.countplot(
    data = df_tmp,
    x = 'rango_etario',
    hue='tiene_alto_valor_adquisitivo',
    ax = ax1,
    palette = ['#cc79a7', '#f0e442']
).set(xticklabels=[])

total = df_tmp.groupby('rango_etario')['tiene_alto_valor_adquisitivo'].count().reset_index()
df_tmp_1 = df_tmp[df_tmp['tiene_alto_valor_adquisitivo']==1].groupby('rango_etario')['tiene_alto_valor_adquisitivo'].count().reset_index()
df_tmp_1['tiene_alto_valor_adquisitivo'] = [i / j * 100 for i,j in zip(df_tmp_1['tiene_alto_valor_adquisitivo'], total['tiene_alto_valor_adquisitivo'])]
total['tiene_alto_valor_adquisitivo'] = [i / j * 100 for i,j in zip(total['tiene_alto_valor_adquisitivo'], total['tiene_alto_valor_adquisitivo'])]

bar1 = sns.barplot(x="rango_etario",  y="tiene_alto_valor_adquisitivo", data=total, color='#cc79a7', ax=ax2)
bar2 = sns.barplot(x="rango_etario", y="tiene_alto_valor_adquisitivo", data=df_tmp_1, color='#f0e442', ax=ax2)

top_bar = mpatches.Patch(color='#ff6961')
bottom_bar = mpatches.Patch(color='#77dd77')

ax1.set_title('Personas con alto poder adquisitivo segun rango etario')
ax1.set_xlabel('')
ax1.set_ylabel('Cantidad')
ax1.legend(['No tiene alto VA', 'Tiene alto VA'], loc='upper right')

ax2.set_xlabel('Rango etario')
ax2.set_ylabel('% Tiene alto valor adquisitivo')


plt.show()

Como podemos ver, es más probable encontrar personas de alto poder adquisitivo entre aquellas que están entre los 30 y 70 años de edad. Mucho menos probable es que esto ocurra para personas de entre 20 y 30 años o más de 70 años, y no ocurrirá para personas menores de 20. Cabe notar que de 70 años en adelante son muy pocas las muestras tomadas.

Por lo que podemos comprobar que efectivamente la edad del encuestado influye sobre la variable objetivo.

### Acción en la bolsa

#### ¿Hay una relación entre invertir en la bolsa y el valor adquisitivo?
Para responder esta pregunta graficaremos la cantidad de gente que accionó en bolsa y la proporción de ellos que poseen un alto poder adquisitivo. Para segmentar a los individuos de la población que estamos estudiando, consideraremos cuatro grupos: por un lado, aquellos que no declararon operaciones en la bolsa argentina, que son aquellos cuyo valor es 0 en esta columna del dataset. Por otro lado, aquellos que informaron pérdidas, identificados por un número menor a 0 en esta columna. En el siguiente gráfico estudiaremos a la población que declaró ganancias, para determinar si existe un valor a partir del cual podemos suponer que una persona tiene poder adquisitivo alto.

In [ ]:
df_tmp = df[['ganancia_perdida_declarada_bolsa_argentina', 'edad']].copy()

sns.set_theme(style="whitegrid")

# Load the diamonds dataset
diamonds = sns.load_dataset("diamonds")

# Plot the distribution of clarity ratings, conditional on carat
sns.displot(
    data=df_tmp,
    x="edad", hue="ganancia_perdida_declarada_bolsa_argentina",
    kind="kde", height=6,
    multiple="fill", clip=(0, None),
    palette="ch:rot=-.25,hue=1,light=.75",
)

In [ ]:
def tiene_mas_ganancia_que(i, row):
    if(row['ganancia_perdida_declarada_bolsa_argentina'] > i):
        return 'Sí'
    return 'No'

In [ ]:
df_tmp = df[['ganancia_perdida_declarada_bolsa_argentina', 'tiene_alto_valor_adquisitivo']].copy()
df_tmp = df_tmp[df_tmp['ganancia_perdida_declarada_bolsa_argentina'] > 0]
i = 0
cats = ['> 0', '> 1000', '> 2000', '> 3000', '> 4000', '> 5000', '> 6000', '> 7000', '> 8000', '> 9000']
for atributo in cats:
    func = functools.partial(tiene_mas_ganancia_que, i)
    df_tmp[atributo] = df_tmp.apply(func, axis=1)
    i += 1000
fig, axes = plt.subplots(nrows=1, ncols=10, dpi=150, figsize = (20, 5), sharey = True)
for i in range(0, 10):
    sns.barplot(
        data=df_tmp,
        x=cats[i],
        y='tiene_alto_valor_adquisitivo',
        ax=axes[i],
        order = ['No', 'Sí'],
        estimator = lambda x: np.mean(x)*100.0,
    )
    axes[i].set_ylabel("")
    axes[i].set_xlabel(cats[i], size=15)
    axes[i].plot(label='big')
    axes[i].set_xticklabels(axes[i].get_xticklabels(), fontsize=10)
    
axes[0].set_yticklabels(axes[0].get_yticks(), fontsize=10)
axes[0].set_ylabel("% Tiene poder adquisitivo alto", size=15)
fig.suptitle("Proporción de gente con alto poder adquisitivo según su ganancia en la bolsa", size=20)

Podemos considerar un buen separador al número 7000, ya que la relación entre personas de alto y bajo poder adquisitivo a partir de este monto es cercana al 100%, mientras que las personas que declararon menos se encuentran en torno al 20%. Por lo tanto, separaremos en las categorías "Mucha ganancia" a quienes hayan declarado un rendimiento en la bolsa mayor a 7000, y "Ganancia" al resto para luego clasificarlos.

In [ ]:
def designar_accion(x):
    if(x > 7000):
        return "Mucha ganancia\n(>7000)"
    elif(x > 0):
        return "Ganancia"
    elif(x == 0):
        return "Nada"
    else:
        return "Perdida"

In [ ]:
df_tmp = df[['ganancia_perdida_declarada_bolsa_argentina', 'tiene_alto_valor_adquisitivo']].copy()
df_tmp['bolsa'] = df['ganancia_perdida_declarada_bolsa_argentina'].apply(designar_accion)
del df_tmp['ganancia_perdida_declarada_bolsa_argentina']
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150, figsize=(7.5 * 2, 4.8))
color_nada = '#B5B5F5'
color_perdida = '#98F585' 
color_MGanancia = '#FA9C8F'
color_ganancia = '#A7F5DA'
labels = df_tmp['bolsa'].value_counts().keys()
ax1.pie(
    x=df_tmp['bolsa'].value_counts(),
    explode=[0.05]*4,
    labels=labels,
    pctdistance=0.5,
    colors = [color_nada, color_perdida, color_MGanancia, color_ganancia]
)
ax1.set_title("Proporción del total según acción en bolsa");
sns.barplot(
    data=df_tmp,
    x='bolsa',
    y='tiene_alto_valor_adquisitivo',
    ax=ax2,
    estimator=lambda x: np.mean(x)*100.0,
    order = ['Mucha ganancia\n(>7000)', 'Ganancia', 'Nada', 'Perdida'],
    palette = [color_MGanancia, color_ganancia, color_nada, color_perdida]
)
ax2.set_ylabel("% Tiene alto valor adquisitivo")
ax2.set_xlabel("")
ax2.set_title("Declaración de acciones en la bolsa")
fig.suptitle('Relación entre las acciones en la bolsa y el poder adquisitivo', fontsize=16)
prom = df['tiene_alto_valor_adquisitivo'].mean()*100.0
ax2.axhline(prom, color="darkred", label="Promedio de personas con alto VA")
ax2.legend()
plt.show()

Parece haber una relación bastante marcada con la gente que declaró un balance neto 0 en acciones de la bolsa y su poder adquisitivo, donde cerca del 80% de estas personas tienen un poder adquisitivo bajo. Lo cual no nos brinda suficiente información pues sabemos que cerca del 76% de las personas no tienen poder adquisitivo alto, es decir, la información es la misma.

Una proporción mucho más destacada se ve entre las personas que declararon una ganancia mayor a \$7000. Siendo que aproximadamente el total de gente que consiguió dicho monto posee poder adquisitivo alto.

También vemos que alrededor de la mitad de las personas que declararon un balance negativo en la bolsa tienen poder adquisitivo alto. Lo cual nos sorpredió un poco, ya que esperabamos que la gente que perdía dinero tenga en general menos poder adqusitivo.
Una posible explicación a este suceso es que, por más que la gente perdió en la bolsa, están accionado. De todas formas como desconocemos el intervalo de tiempo para el que la muestra fue realizada (es decir, si es por día, semana, mes, anual, década, etc) no podemos inferir sobre las previas acciones en bolsa del mismo grupo de personas.

Un dato que nos sorprendió es que la gente que accionó en bolsa, pero sin embargo no ganó mucho dinero en ella, no tiene poder adquisitivo alto, a diferencia de las personas que tienen pérdidas cuya proporción es casi el doble.
Podríamos atribuir esta característica a los tipos de acciones que realizan estos individuos, ya que, es esperable que las personas que accionan sobre activos más riesgosos obtengan mayor beneficio a costa del riesgo, mientras que aquellas personas que accionan sobre activos seguros su ganancia será despreciable frente al resto.


Finalmente, concluimos que hay una clara relación entre la declaración de ganancias superiores a \$7000 y el poder adquisitivo de las personas que lo hicieron, donde casi la totalidad de ellas tienen un poder adquisitivo algo.

#### Género y acción en la bolsa
Veamos ahora la relación que existe entre el género y la acción en la bolsa. ¿Qué diferencia hay entre hombres y mujeres en general? ¿Hay más hombres que declaran ganancia en la bolsa? ¿Hay más mujeres que declaran pérdidas en la bolsa?
¿Si declaran ganancias, existe una diferencia amplia entre el monto de una mujer y de un hombre? ¿Y si declaran pérdidas?

In [ ]:
df_tmp = df.copy()
df_tmp = df_tmp[df_tmp['ganancia_perdida_declarada_bolsa_argentina'] > 0]
df_tmp['ganancia_perdida_declarada_bolsa_argentina'] = df_tmp['ganancia_perdida_declarada_bolsa_argentina'].apply(math.log10)
df_tmp2 = df.copy()
df_tmp2 = df_tmp2[df_tmp2['ganancia_perdida_declarada_bolsa_argentina'] < 0]
df_tmp2['ganancia_perdida_declarada_bolsa_argentina'] = df_tmp2['ganancia_perdida_declarada_bolsa_argentina'].apply(lambda x: math.log10(-1.0*x))
df_tmp3 = df.copy()
df_tmp3 = df_tmp3[df_tmp3['ganancia_perdida_declarada_bolsa_argentina'] != 0]
df_tmp3['bolsa'] = df_tmp3['ganancia_perdida_declarada_bolsa_argentina'].apply(lambda x: "Ganancia" if x > 0 else "Pérdida")
f, axs = plt.subplots(nrows=2, ncols=2,figsize=(16, 10), dpi=150)
gs = axs[0, 1].get_gridspec()
# remove the underlying axes
for ax in axs[0, 0:]:
    ax.remove()
axbig = f.add_subplot(gs[0, 0:])
ax3 = axs[1, 0]
ax4 = axs[1, 1]
sns.histplot(
    df_tmp3,
    x="bolsa", hue="genero",
    ax=axbig,
    multiple="stack",
    palette=("#F1F1F2", "#B5B5F5"),
    edgecolor=".3",
    linewidth=.5,
    shrink=0.7,
)
sns.boxplot(data=df_tmp,
            x='genero', 
            y='ganancia_perdida_declarada_bolsa_argentina', 
            palette=("#F1F1F2", "#B5B5F5"),
            ax=ax3,
            fliersize=1,
            linewidth=2,)
sns.boxplot(data=df_tmp2,
            x='genero',
            y='ganancia_perdida_declarada_bolsa_argentina', 
            palette=("#F1F1F2", "#B5B5F5"),
            ax=ax4,
            fliersize=1,
            linewidth=2,)
plt.suptitle('Relación entre género y acción en bolsa', size=24)
axbig.set_xlabel("")
axbig.set_xticklabels("")
axbig.set_ylabel("Cantidad encuestada", size=16)
ax3.set_title("Ganancia", size=20)
ax3.set_ylabel("Declaración\n Escala logarítmica en base 10", size=16)
ax3.set_xlabel("")
ax3.set_xticklabels(ax3.get_xticklabels(), size=14)
ax3.set_yticklabels(ax3.get_yticks(), size=14)
ax4.set_title("Pérdida", size=20)
ax4.set_ylabel("")
ax4.set_xlabel("")
ax4.set_xticklabels(ax4.get_xticklabels(), size=14)
ax4.set_yticklabels(ax4.get_yticks(), size=14)
ax4.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
sns.despine(left=True, bottom=True)

De estos gráficos podemos obtener las respuestas:
* Efectivamente hay más hombres que declaran tanto ganancias como pérdidas en la bolsa. Sin embargo, la cantidad de hombres y de mujeres en el dataset es desproporcionada, por lo que, es lógico obtener estos resultados.
* **Ganancia:** podemos ver que el monto declarado en la bolsa de hombres y mujeres difiere. Si bien hay mucha intersección de los intervalos intercuartílicos, podemos ver que las medianas de los boxplots están desfasadas, y observar que el cuartil 75% para los hombres se encuentra muy por encima del cuartil 75% de las mujeres, que incluso es similar a la mediana para los hombres.
* **Pérdida:** no vemos una diferencia marcada, se comportan similar.

### Barrio

#### ¿Hay una relación entre el barrio y el valor adquisitivo?
Veamos la distribución de las ciudades y cómo esta afecta al poder adquisitivo. Consideraremos estadísticamente significativos aquellos barrios de los que se hayan tomado al menos 100 muestras.

In [ ]:
fig, (ax2, ax1) = plt.subplots(nrows=1, ncols=2, dpi=400, figsize=(8.5 * 2, 6))

cant_min = 100
tabla = df['barrio'].value_counts()
barrios_significativos = tabla[tabla > cant_min].reset_index()['index'].unique()
df_tmp = df[['barrio', 'tiene_alto_valor_adquisitivo']].copy()
df_tmp['barrio'] = df_tmp['barrio'].apply(lambda x: x if x in barrios_significativos else 'Otro')


paleta=sns.color_palette("deep")

barrios = df_tmp['barrio'].value_counts()
squarify.plot(barrios, label=barrios.keys(), alpha=0.75,color=paleta,  text_kwargs={'fontsize':7}, norm_x=1, norm_y=3, ax=ax1)
ax1.axis('off')

orden = df_tmp.groupby(['barrio'])['tiene_alto_valor_adquisitivo'].aggregate(lambda x : len(x)).reset_index().sort_values('tiene_alto_valor_adquisitivo', ascending = False)
sns.barplot(
    data=df_tmp,
    palette=paleta,
    x='barrio',
    y='tiene_alto_valor_adquisitivo',
    order=orden['barrio'],
    ax=ax2,
    estimator=lambda x: np.mean(x)*100.0
)

ax1.set_title("Proporción de personas registradas según barrio")
ax1.set_xlabel("Significancia mínima: 100 muestras")

ax2.set_ylabel("% Tiene alto valor adquisitivo")
ax2.set_xlabel("Barrio")
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90)
ax2.set_title('Relación entre barrio y poder adquisitivo')
prom = df['tiene_alto_valor_adquisitivo'].mean()*100.0
ax2.axhline(prom, color="red", label="Promedio de personas con alto VA")
ax2.legend()

plt.show()

Aquí vemos que no hay una tendencia clara que nos permita segmentar a las personas según el barrio en el que viven y a partir de eso inferir si su poder adquisitivo es alto o bajo. Además, vemos que una enorme cantidad de las muestras que estamos analizando provienen del barrio de Palermo, lo que introduciría un sesgo muy fuerte si tomáramos decisiones teniendo en cuenta esta columna. Podríamos, por ejemplo, decir que las personas que viven en Belgrano y en Agronomía tienen una alta probabilidad de tener bajo poder adquisitivo, pero eso nos llevaría al overfitting en nuestro modelo, pues consideramos que no hay suficiente significancia estadística.

### Años estudiados

#### ¿Hay una relación entre la cantidad de años estudiados y el valor adquisitivo?
A priori es de esperar que a mayores estudios alcanzados mayores proporciones de poder adquisitivo alto.

Veamos si encontramos una relación entre estas dos variables.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150, figsize=(7.5 * 2, 4.8))

df_tmp = df[['anios_estudiados', 'tiene_alto_valor_adquisitivo']].copy()

ax1.set_title("")
sns.violinplot(
    data=df_tmp,
    x='tiene_alto_valor_adquisitivo',
    y='anios_estudiados',
    ax=ax1,
    color="#3268ca"
)
ax1.tick_params(which='both', width=2)
ax1.set_ylabel("Años estudiados")
ax1.set_xlabel("¿Tiene alto valor adquisitivo?")
ax1.set_xticklabels(labels=["No", "Sí"])

sns.barplot(
    data=df_tmp,
    color="#3268ca",
    x='anios_estudiados',
    y='tiene_alto_valor_adquisitivo',
    ax=ax2,
    estimator = lambda x: np.mean(x)*100.0
)

ax2.set_ylabel("% Tiene alto valor adquisitivo")
ax2.set_xlabel("Años estudiados")
ax2.set_title('')
fig.suptitle('Relación entre la cantidad de años estudiados y poder adquisitivo', fontsize=16)
prom = df['tiene_alto_valor_adquisitivo'].mean()*100.0
ax2.axhline(prom, color="lightblue", label="Promedio de personas con alto VA")
ax2.legend()

plt.show()

Podemos ver una relación muy marcada entre estas dos variables, donde quienes tienen 12 años de estudio o menos se encuentran entre la población de menor poder adquisitivo, y quienes superan los 18 años de estudio tienden a tener un poder adquisitivo alto. El violin plot de la izquierda, a su vez, nos ayuda a ver que los dos grupos antes mencionados cuentan con pocos individuos presentes en la muestra, mientras que el conjunto que tiene entre 13 y 17 años de estudios concentra a la mayor parte de la población muestreada.

Además, el gráfico de la izquierda nos muestra la distribuciones de años estudiados para las personas con alto y bajo poder adquisitivo. Como se puede ver, las personas con mayor poder adquisitivo tienden a tener mayor cantidad de años estudiados.

Concluimos que esta variable afecta significativamente a nuestra variable objetivo.

### Horas trabajadas

Veamos, inicialmente, cómo se distribuye esta variable en el dataset.

In [ ]:
df_tmp = df[['horas_trabajo_registradas']].copy()
plt.figure(dpi=100)
sns.displot(df_tmp, x='horas_trabajo_registradas', kind="kde", fill=True)
plt.show()

Vemos una distribución con muy poca densidad en las colas, lo cual es lógico por tratarse de horas de trabajo, y un aumento drástico en las 40 horas semanales de trabajo. Esto es muy razonable debido a que 40 horas semanales de trabajo se corresponde con trabajar 8 horas 5 días por semana, que es la jornada de trabajo más habitual en la región en la que se tomaron las muestras.

#### ¿Hay una relación entre la cantidad de horas trabajadas y el valor adquisitivo?
En principio esperamos ver relación. A pocas horas trabajadas esperamos poco índice de poder adquisitivo alto.

In [ ]:
df_tmp = df[['horas_trabajo_registradas', 'tiene_alto_valor_adquisitivo']].copy()
df_tmp['rango_horas_trabajadas'] = pd.cut(df_tmp.horas_trabajo_registradas, range(10, int(df_tmp['horas_trabajo_registradas'].max()) + 10, 10))

# Configuro el grafico, armo el countplot y setteo labels varios
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, dpi=150, figsize=(6.3 * 2, 4 * 2))
sns.countplot(
    data = df_tmp,
    x = 'rango_horas_trabajadas',
    hue='tiene_alto_valor_adquisitivo',
    ax = ax1,
    palette = ['#cc79a7', '#f0e442']
).set(xticklabels=[])

total = df_tmp.groupby('rango_horas_trabajadas')['tiene_alto_valor_adquisitivo'].count().reset_index()
df_tmp_1 = df_tmp[df_tmp['tiene_alto_valor_adquisitivo']==1].groupby('rango_horas_trabajadas')['tiene_alto_valor_adquisitivo'].count().reset_index()
df_tmp_1['tiene_alto_valor_adquisitivo'] = [i / j * 100 for i,j in zip(df_tmp_1['tiene_alto_valor_adquisitivo'], total['tiene_alto_valor_adquisitivo'])]
total['tiene_alto_valor_adquisitivo'] = [i / j * 100 for i,j in zip(total['tiene_alto_valor_adquisitivo'], total['tiene_alto_valor_adquisitivo'])]

bar1 = sns.barplot(x="rango_horas_trabajadas",  y="tiene_alto_valor_adquisitivo", data=total, color='#cc79a7', ax=ax2)
bar2 = sns.barplot(x="rango_horas_trabajadas", y="tiene_alto_valor_adquisitivo", data=df_tmp_1, color='#f0e442', ax=ax2)

top_bar = mpatches.Patch(color='#ff6961')
bottom_bar = mpatches.Patch(color='#77dd77')

ax1.set_title('Personas con alto poder adquisitivo segun la cantidad de horas trabajadas')
ax1.set_xlabel('')
ax1.set_ylabel('Cantidad')
ax1.legend(['No tiene alto VA', 'Tiene alto VA'], loc='upper right')

ax2.set_xlabel('Horas trabajadas')
ax2.set_ylabel('% Tiene alto valor adquisitivo')


plt.show()

Vemos que en los casos en que las horas trabajadas son menores o iguales a 30 casi todas las personas muestreadas tienen un poder adquisitivo bajo. Los individuos con poder adquisitivo alto empiezan a aparecer en su mayoría a partir de las 30 horas trabajadas, y para 40 horas en adelante empiezan a ocupar un porcentaje importante de la muestra. Por ejemplo, para quienes trabajan entre 50 y 60 horas, la relación es casi 40/60.

Sin embargo, también es necesario ver en el gráfico superior, que la significancia estadística de las colas es prácticamente nula.

### ¿Influye el rol familiar registrado sobre el poder adquisitivo? ¿Las personas con hijos tienen menos poder adquisitivo?

In [ ]:
df_tmp = df[['rol_familiar_registrado', 'tiene_alto_valor_adquisitivo']].copy()

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150, figsize=(7.5 * 2, 4.8))

color_sinFam = '#B5B5F5'
color_casado = '#98F585' 
color_conHijos = '#FA9C8F'
color_soltero = '#A7F5DA'
color_otro = '#FAD566'

labels = df_tmp['rol_familiar_registrado'].value_counts().keys()
ax1.pie(
    x=df_tmp['rol_familiar_registrado'].value_counts(),
    explode=[0.01]*len(df_tmp['rol_familiar_registrado'].value_counts()),
    labels=labels,
    pctdistance=0.5,
    colors = [color_casado, color_sinFam, color_conHijos, color_soltero, color_otro]
)

ax1.set_title("Proporción del total según rol familiar registrado");
sns.barplot(
    data=df_tmp,
    y='rol_familiar_registrado',
    x='tiene_alto_valor_adquisitivo',
    ax=ax2,
    estimator=lambda x: np.mean(x)*100.0,
    order = ['casado', 'sin_familia', 'soltero_a', 'otro','con_hijos'],
    palette = [color_casado, color_sinFam, color_soltero, color_otro, color_conHijos]
)

ax2.set_ylabel("Rol familiar registrado")
ax2.set_xlabel("% Tiene alto valor adquisitivo")
ax2.set_title('Categorías y su poder adquisitivo')
fig.suptitle('Relación entre el rol familiar registrado y el poder adquisitivo', fontsize=16)
prom = df['tiene_alto_valor_adquisitivo'].mean()*100.0
ax2.axvline(prom, color="darkred", label="Promedio de personas con alto VA")
ax2.legend()
plt.show()

De este gráfico notamos lo siguiente:
* La categoría 'casado' prácticamente duplica la cantidad promedio de personas con poder adquisitivo alto. Sin embargo, por sí sola no nos brinda una regla de decisión.
* Vemos que en general las personas sin familia, solteros/as, con hijos y otros tienden a tener un poder adquisitivo bajo.

Pudimos notar que efectivamente, las personas que tienen hijos tienden fuertemente a tener bajo poder adquisitivo.

Podríamos argumentar que esto se debe a los gastos en manutención. Mientras que por contraparte los casados (que no tienen hijos) serán dos personas cuyos sueldos podrían incluso compartir sin necesidad de mantener un tercero (o más).

#### ¿La religión afecta al poder adquisitivo? ¿El género? ¿Afectan en conjunto?

Para empezar, veamos cómo se distribuyen la religión y el género en nuestro dataset.

In [ ]:
df_tmp = df[['genero', 'religion']].copy()
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150, figsize=(7.5 * 2, 4.8))

color_crist = '#B5B5F5'
color_jud = '#98F585' 
color_bud = '#FA9C8F'
color_ateo = '#A7F5DA'
color_otro = '#FAD566'

ax1.pie(
    x=df_tmp['genero'].value_counts(),
    explode=[0.01]*len(df_tmp['genero'].value_counts()),
    labels=df_tmp['genero'].value_counts().keys(),
    pctdistance=0.5,
    autopct='%1.0f%%',
    colors = ['#cc79a7', '#f0e442']
)
ax1.set_title("Proporciones del atributo género");
ax2.pie(
    x=df_tmp['religion'].value_counts(),
    explode=[0.05]*len(df_tmp['religion'].value_counts()),
    labels=df_tmp['religion'].value_counts().keys(),
    pctdistance=0.5,
    colors = [color_crist, color_jud, color_bud, color_ateo, color_otro]
)

ax2.set_title('Proporciones del atributo religión')
fig.suptitle('Distribuciones de los atributos género y religión', fontsize=16)
plt.show()


In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150, figsize=(7.5*2, 4.8), sharey=True)
df_tmp_h = df[df['genero'] == 'hombre']
orden = df_tmp_h.groupby(['religion'])['tiene_alto_valor_adquisitivo'].aggregate(lambda x : np.mean(x)).reset_index().sort_values('tiene_alto_valor_adquisitivo', ascending = False)
sns.barplot(
    data=df_tmp_h,
    palette="pastel",
    x='religion',
    y='tiene_alto_valor_adquisitivo',
    ax=ax1,
    order=orden['religion'],
    estimator=lambda x: np.mean(x)*100.0
)
prom = df_tmp_h['tiene_alto_valor_adquisitivo'].mean()*100.0
ax1.set_ylabel("% Tiene alto valor adquisitivo")
ax1.set_xlabel("Religion")
ax1.set_title('Sólo hombres')
ax1.axhline(prom, color="darkred", label="Promedio de hombres con alto VA")
ax1.legend()
df_tmp_m = df[df['genero'] == 'mujer']
orden = df_tmp_m.groupby(['religion'])['tiene_alto_valor_adquisitivo'].aggregate(lambda x : np.mean(x)).reset_index().sort_values('tiene_alto_valor_adquisitivo', ascending = False)
sns.barplot(
    data=df_tmp_m,
    palette="pastel",
    x='religion',
    y='tiene_alto_valor_adquisitivo',
    ax=ax2,
    order=orden['religion'],
    estimator=lambda x: np.mean(x)*100.0
)
prom = df_tmp_m['tiene_alto_valor_adquisitivo'].mean()*100.0
ax2.set_ylabel("")
ax2.set_xlabel("Religion")
ax2.set_title('Sólo mujeres')
ax2.axhline(prom, color="darkred", label="Promedio de mujeres con alto VA")
ax2.legend()
fig.suptitle('Relación entre religión, género y poder adquisitivo', fontsize=16)
plt.show()

Como primera observación del gráfico tenemos la línea horizontal que marca el promedio de personas con poder adquisitivo alto según el género. Podemos ver como las mujeres tienen un promedio del 10% mientras que los hombres el promedio asciende hasta el 30%.
Es decir, vemos que efectivamente el género afecta el poder adquisitivo.

Siguiendo con el análisis, no encontramos elementos en esta variable que nos lleven a clasificar a los miembros de alguna religión en particular como personas de alto poder adquisitivo, pero sí podemos observar muy pocas mujeres que profesen el judaísmo u otras religiones y tengan alto poder adquisitivo.

Veamos además, que para las religiones que tienden a tener menor poder adquisitivo, el intervalor de confianza del barplot es mucho mayor, esto quiere decir que la significancia estadística de los datos es pobre.

#### Género, rango etario y cantidad de horas trabajadas 
Queremos observar si existe algún tipo de relación entre la edad, el género y la cantidad de horas trabajadas. ¿Cambia según la edad? ¿Cambia según el género? Si lo hace ¿A qué se podría deber?

In [ ]:
df_tmp = df.copy()
df_tmp['rango_edad'] = pd.cut(df_tmp.edad, range(15, df_tmp['edad'].max() + 10, 10))
f, ax = plt.subplots(figsize=(16, 10), dpi=150)
# Draw a violinplot with a narrower bandwidth than the default
sns.boxplot(data=df_tmp,
            x='rango_edad', 
            y='horas_trabajo_registradas', 
            color="#B5B5F5", 
            ax=ax,
            hue='genero',
            fliersize=1,
            linewidth=2,
            medianprops=dict(color="orange", alpha=0.7))
ax.set_ylabel("Horas de trabajo registrado", size=20)
ax.set_xlabel("Rango etario", size=20)
ax.set_xticklabels(ax.get_xticklabels(), size=15)
ax.set_yticklabels(ax.get_yticks(), size=15)
ax.set_title("Relación entre la cantidad de horas trabajadas y la edad", size=24)
plt.legend(fontsize=20)
sns.despine(left=True, bottom=True)

De este gráfico podemos observar y responder lo siguiente:
* En general, la edad no influye mucho en la cantidad de horas trabajadas, obviando casos de gente joven (15, 25] años y los ancianos (65,95] la distribución se mantuvo constante
* En los extremos la distribución cambió, a medida que más longeva sea la persona, menos cantidad de horas trabaja.
* El último boxplot se deberá analizar con cuidado ya que hay muy pocas muestras.
* En general, el género sí parece afecta a la cantida de horas trabajadas, en todos los casos, el intervalo intercuartílico para los hombres está más elevado que el de las mujeres.
* de todas formas, dicha diferencia no es muy marcada ya que, para ambos géneros, la mayor parte de la población trabaja 40 horas semanales (esto se observa en la mediana que está en un extremo del boxplot y coincide para casi todos ellos).

### ¿El rol familiar registrado y el estado marital tienen la misma influencia sobre el poder adquisitivo?
Esto es, ¿obtener información de una de esas variables da información sobre la otra? Veámoslo en un heatmap.

In [ ]:
plt.figure(dpi = 160)
df_tmp = df[['estado_marital','rol_familiar_registrado', 'tiene_alto_valor_adquisitivo']].copy()
cooccurrence_g = pd.pivot_table(
    df_tmp,
    'tiene_alto_valor_adquisitivo',
    'estado_marital',
    'rol_familiar_registrado',
    'count'
)
sns.heatmap(cooccurrence_g, cmap='Wistia', annot=True, fmt=".5g")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.ylabel("Estado marital")
plt.xlabel("Rol familiar registrado")
plt.title("Relación entre el estado marital junto al rol familiar registrado con el poder adquisitivo")
plt.show()

De las celdas con datos faltantes y las cantidades observadas podemos deducir que, efectivamente, en términos generales la información obtenida de ambas variables es equivalente. Pues podemos ver sobre la distribución del heatmap que de las 35 celdas posibles, sólo 5 continen a casi el 90% de las muestras.

### Trabajo y categoría de trabajo

Veamos qué resulta de intersecar las categorías de las variables `trabajo` y `categoria_de_trabajo` , visualizándolas 

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=200, figsize=(7*2, 5))
df_tmp = df[['trabajo','categoria_de_trabajo', 'tiene_alto_valor_adquisitivo']].copy()
cooccurrence_count = pd.pivot_table(
    df_tmp,
    'tiene_alto_valor_adquisitivo',
    'trabajo',
    'categoria_de_trabajo',
    'count'
)
cooccurrence_mean = pd.pivot_table(
    df_tmp,
    'tiene_alto_valor_adquisitivo',
    'trabajo',
    'categoria_de_trabajo',
    'mean'
)
sns.heatmap(cooccurrence_count, cmap='Wistia', annot=True, fmt=".5g", ax=ax1)
sns.heatmap(cooccurrence_mean, cmap='Wistia', annot=True, fmt=".2g", ax=ax2).set(yticklabels=[])
ax1.set_ylabel("Trabajo")
ax1.set_xlabel("Categoría de trabajo")
ax1.set_title("Cantidad de personas")
ax2.set_ylabel("")
ax2.set_xlabel("Categoría de trabajo")
ax2.set_title("Personas de dichas facultades que poseen poder adquisitivo alto.")
fig.suptitle('Relación entre categoría de trabajo, trabajo y poder adquisitivo', fontsize=16)
plt.show()

De aquí se pueden observar relaciones interesantes entre las variables y cómo estas afectan el poder adquisitivo. Sin embargo, decidimos desestimar a priori esta información pues, en caso de usarla, creemos que será muy específico sobre datos que en general no tienen suficiente significancia estadística.
Como se puede ver en el gráfico de la izquierda, la mayor parte de la gente está declarada como 'En relación de dependencia' y, como se puede ver en el gráfico de la derecha, dicha relación no nos ayuda a encontrar personas con alto poder adquisitivo, pues sus valores son de caracter fluctuante.

## Baseline

##### Construyendo el Baseline
Iremos particionando el dominio en base a los resultados obtenidos en el análisis exploratorio.

La primera cualidad observada del análisis de los datos es que casi la totalidad de la gente que obtuvo una ganancia superior a 7000 en la bolsa, tiene poder adquisitivo alto.

Por lo cual, nos quedaremos para analizar sólo a aquellos que no ganaron más de 7000 en la bolsa. Mientras que sobre los demás (es decir > 7000 en ganancias) diremos que poseen poder adquisitivo alto. 

In [ ]:
df_pipeline = df[df['ganancia_perdida_declarada_bolsa_argentina'] < 7000]

La segunda cualidad que observamos en el análisis de datos sobre el data frame original fue la edad, cuyo valor condicionaba de la siguiente forma:
* Si tenés menos de 30 años o más de 70 años, todo parece indicar que no posees un alto valor adquisitivo
* En cambio, si te encontrás en el rango intermedio, que tengas valor adquisitivo alto dependerá de otras cualidades

Veamos si esto se mantiene en nuestro data frame actual, para ello graficaremos lo mismo pero con los datos recortados

In [ ]:
df_tmp = df_pipeline[['edad', 'tiene_alto_valor_adquisitivo']].copy()
df_tmp['rango_etario'] = pd.cut(df_tmp.edad, range(10, int(df_tmp['edad'].max()) + 10, 10))

# Configuro el grafico, armo el countplot y setteo labels varios
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, dpi=150, figsize=(6.3 * 2, 4 * 2))
sns.countplot(
    data = df_tmp,
    x = 'rango_etario',
    hue='tiene_alto_valor_adquisitivo',
    ax = ax1,
    palette = ['#cc79a7', '#f0e442']
).set(xticklabels=[])

total = df_tmp.groupby('rango_etario')['tiene_alto_valor_adquisitivo'].count().reset_index()
df_tmp_1 = df_tmp[df_tmp['tiene_alto_valor_adquisitivo']==1].groupby('rango_etario')['tiene_alto_valor_adquisitivo'].count().reset_index()
df_tmp_1['tiene_alto_valor_adquisitivo'] = [i / j * 100 for i,j in zip(df_tmp_1['tiene_alto_valor_adquisitivo'], total['tiene_alto_valor_adquisitivo'])]
total['tiene_alto_valor_adquisitivo'] = [i / j * 100 for i,j in zip(total['tiene_alto_valor_adquisitivo'], total['tiene_alto_valor_adquisitivo'])]

bar1 = sns.barplot(x="rango_etario",  y="tiene_alto_valor_adquisitivo", data=total, color='#cc79a7', ax=ax2)
bar2 = sns.barplot(x="rango_etario", y="tiene_alto_valor_adquisitivo", data=df_tmp_1, color='#f0e442', ax=ax2)

top_bar = mpatches.Patch(color='#ff6961')
bottom_bar = mpatches.Patch(color='#77dd77')

ax1.set_title('Personas con alto poder adquisitivo segun rango etario')
ax1.set_xlabel('')
ax1.set_ylabel('Cantidad')
ax1.legend(['No tiene alto VA', 'Tiene alto VA'], loc='upper right')

ax2.set_xlabel('Rango etario')
ax2.set_ylabel('% Tiene alto valor adquisitivo')


plt.show()

Observamos que efectivamente se puede seguir obteniendo la misma información de dichos gráficos.


Por lo que, nos quedaremos con las personas cuyo rango etario esté entre 30 y 70 años, y sobre aquellas que no, diremos que tienen poder adquisitivo bajo.

In [ ]:
df_pipeline = df_pipeline[df_pipeline['edad'].isin(range(31, 71))]

De la información recolectada en el análisis exploratorio pudimos observar como el estado marital influye tremendamente en la variable objetivo, siendo que tener hijos quita a la mayor parte de la población de tener un alto poder adquisitivo, asímismo también lo hacen si tenés 'otro' o 'soltero_a'.

Revisemos si estas afirmaciones siguen siendo válidas para la gente entre 30 y 70 años que no accionó con mucho éxito en la bolsa.

In [ ]:
df_tmp = df_pipeline[['rol_familiar_registrado', 'tiene_alto_valor_adquisitivo']].copy()

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150, figsize=(7.5 * 2, 4.8))

color_sinFam = '#B5B5F5'
color_casado = '#98F585' 
color_conHijos = '#FA9C8F'
color_soltero = '#A7F5DA'
color_otro = '#FAD566'

labels = df_tmp['rol_familiar_registrado'].value_counts().keys()
ax1.pie(
    x=df_tmp['rol_familiar_registrado'].value_counts(),
    explode=[0.01]*len(df_tmp['rol_familiar_registrado'].value_counts()),
    labels=labels,
    pctdistance=0.5,
    colors = [color_casado, color_sinFam, color_conHijos, color_soltero, color_otro]
)

ax1.set_title("Proporción del total según rol familiar registrado");
sns.barplot(
    data=df_tmp,
    y='rol_familiar_registrado',
    x='tiene_alto_valor_adquisitivo',
    ax=ax2,
    estimator=lambda x: np.mean(x)*100.0,
    order = ['casado', 'sin_familia', 'soltero_a', 'otro','con_hijos'],
    palette = [color_casado, color_sinFam, color_soltero, color_otro, color_conHijos]
)

ax2.set_ylabel("Rol familiar registrado")
ax2.set_xlabel("% Tiene alto valor adquisitivo")
ax2.set_title('Categorías y su poder adquisitivo')
fig.suptitle('Relación entre el rol familiar registrado y el poder adquisitivo', fontsize=16)
prom = df_pipeline['tiene_alto_valor_adquisitivo'].mean()*100.0
ax2.axvline(prom, color="darkred", label="Promedio de personas con alto VA")
ax2.legend()
plt.show()

Por lo que se puede observar que se mantiene lo dicho anteriormente, es decir, la gente que no está casada, tiene altas chances de no tener poder adquisitivo alto.

Por lo que, estimaremos que tienen poder adquisitivo bajo y procedemos a estudiar sólo a aquellos que han contraído matrimonio: 

In [ ]:
df_pipeline = df_pipeline[df_pipeline['rol_familiar_registrado'] == 'casado']

Veamos cómo evolucionó la variable de ganancia/perdida declarada en la bolsa.

In [ ]:
df_tmp = df_pipeline[['ganancia_perdida_declarada_bolsa_argentina', 'tiene_alto_valor_adquisitivo']].copy()
df_tmp['bolsa'] = df_tmp['ganancia_perdida_declarada_bolsa_argentina'].apply(designar_accion)
del df_tmp['ganancia_perdida_declarada_bolsa_argentina']
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150, figsize=(7.5 * 2, 4.8))
color_nada = '#B5B5F5'
color_perdida = '#98F585' 
color_ganancia = '#A7F5DA'
labels = df_tmp['bolsa'].value_counts().keys()
ax1.pie(
    x=df_tmp['bolsa'].value_counts(),
    explode=[0.05]*3,
    labels=labels,
    pctdistance=0.5,
    colors = [color_nada, color_perdida, color_ganancia]
)
ax1.set_title("Proporción del total según acción en bolsa");
sns.barplot(
    data=df_tmp,
    x='bolsa',
    y='tiene_alto_valor_adquisitivo',
    ax=ax2,
    estimator=lambda x: np.mean(x)*100.0,
    order = ['Ganancia', 'Nada', 'Perdida'],
    palette = [ color_ganancia, color_nada, color_perdida]
)
ax2.set_ylabel("% Tiene alto valor adquisitivo")
ax2.set_xlabel("Declaración de acciones en la bolsa")
ax2.set_title('Declaracion en la bolsa')
fig.suptitle('Relación entre las acciones en la bolsa y el poder adquisitivo', fontsize=16)
prom = df_pipeline['tiene_alto_valor_adquisitivo'].mean()*100.0
ax2.axhline(prom, color="darkred", label="Promedio de personas con alto VA")
ax2.legend()
plt.show()

Vemos que se pueden filtrar aún suficiente cantidad de gente cuyo valor adquisitivo es alto, estos son aquellos que declararon pérdidas en la bolsa.

Por lo tanto, sobre aquellos que han perdido en la bolsa, diremos que poseen poder adquisitivo alto, mientras que, seguiremos analizando la información de las personas con ganancias o que no accionaron en la bolsa.

In [ ]:
df_pipeline = df_pipeline[df_pipeline['ganancia_perdida_declarada_bolsa_argentina'] >= 0]

Otra cualidad que vimos interesante para predecir la variable objetivo se trata de la cantidad de años estudiados, veamos cómo se comporta a esta altura:

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=150, figsize=(7.5 * 2, 4.8))

df_tmp = df_pipeline[['anios_estudiados', 'tiene_alto_valor_adquisitivo']].copy()

ax1.set_title("")
sns.violinplot(
    data=df_tmp,
    x='tiene_alto_valor_adquisitivo',
    y='anios_estudiados',
    ax=ax1,
    color="#3268ca"
)
ax1.tick_params(which='both', width=2)
ax1.set_ylabel("Años estudiados")
ax1.set_xlabel("¿Tiene alto valor adquisitivo?")
ax1.set_xticklabels(labels=["No", "Sí"])

sns.barplot(
    data=df_tmp,
    color="#3268ca",
    x='anios_estudiados',
    y='tiene_alto_valor_adquisitivo',
    ax=ax2,
    estimator = lambda x: np.mean(x)*100.0
)

ax2.set_ylabel("% Tiene alto valor adquisitivo")
ax2.set_xlabel("Años estudiados")
ax2.set_title("")
fig.suptitle('Relación entre la cantidad de años estudiados y poder adquisitivo', fontsize=16)
prom = df_pipeline['tiene_alto_valor_adquisitivo'].mean()*100.0
ax2.axhline(prom, color="lightblue", label="Promedio de personas con alto VA")
ax2.legend()

plt.show()

Por lo que, clasificaremos como bajo poder adquisitivo a aquella gente que tuvo 12 o menos años estudiados, y en contrapartida clasificaremos como alto poder adquisitivo a aquella gente que tuvo 17 o más años estudiados.

Por lo que:

In [ ]:
df_pipeline = df_pipeline[(df_pipeline['anios_estudiados'] >= 13) & (df_pipeline['anios_estudiados'] <= 16)]

Ya estando a esta altura, nos quedaría por analizar sólo un 20% de las muestras. Pero no encontramos una variable que nos proporcione información útil a esta altura. Vimos que las condiciones apartir de esta línea o eran demasiado específicas, o no eran buenas reglas de decisión pues sólo mejoraban en a lo sumo un 0.2% de acierto.

Es de esta forma que cerramos en análisis de los datos, y procederemos a contruir el código del baseline y así observar cómo se comporta con la muestra de entrenamiento.



##### El baseline

In [ ]:
def baseline_aux(row):
    if(row['ganancia_perdida_declarada_bolsa_argentina'] > 7000):
        return 1
    if(row['edad'] <= 30 or row['edad'] > 70):
        return 0
    if(row['rol_familiar_registrado'] != 'casado'):
            return 0
    if(row['ganancia_perdida_declarada_bolsa_argentina'] < 0):
                return 1
    if(row['anios_estudiados'] >= 17):
                    return 1
    if(row['anios_estudiados'] <= 12):
                    return 0
    return 0

def baseline(df):
    return df.apply(baseline_aux, axis=1)

In [ ]:
df = feature_engineering(get_data())
i = np.mean(df['tiene_alto_valor_adquisitivo'] == baseline(df))

print(f"El índice de aciertos total fue de: {round((i*100), 5)}%")

## Bonus Track

Asumiendo que la distribución de personas con poder adquisitivo alto se mantiene en un 76% vs 24%, nuestro baseline tiene las siguientes características:

In [ ]:
df_0 = df[df['tiene_alto_valor_adquisitivo'] == 0]
print(f"Predicción de ceros: {round(np.mean(baseline(df_0) == 0)*100,0)}%")
df_1 = df[df['tiene_alto_valor_adquisitivo'] == 1]
print(f"Predicción de unos: {round(np.mean(baseline(df_1) == 1)*100,0)}%")

P("Tiene bajo VA"|"Predecimos bajo VA") = $0.86$

P("Tiene alto VA"|"Predecimos alto VA") = $0.76$


###### Este resultado se obtiene utilizando Bayes

$p_0$: "Predecir bajo poder adquisitivo"

$p_1$: "Predecir alto poder adquisitivo"

$s_0$: "Tiene bajo poder adquisitivo"

$s_1$: "Tiene alto poder adquisitivo"


$$ P(s_0|p_0) = \frac{P(p_0|s_0)P(s_0)}{P(p_0)} = \frac{0.95 \times 0.76}{0.95\times0.76 + 0.49 \times0.24} = 0.86$$

$$ P(s_1|p_1) = \frac{P(p_1|s_1)P(s_1)}{P(p_1)} = \frac{0.51 \times 0.24}{0.51\times0.24 + 0.05 \times0.76} = 0.76$$